<a href="https://colab.research.google.com/github/Howl06/practice/blob/main/Naive_bayes_poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
traindf = pd.read_csv("/content/drive/MyDrive/naive_b_news/poem_train.csv", encoding="utf-8")
traindf

In [ ]:
testdf = pd.read_csv("/content/drive/MyDrive/naive_b_news/poem_test.csv", encoding="utf-8")
testdf

In [ ]:
!pip install jieba

In [ ]:
import jieba
a = "青天有月來幾時"
" ".join(jieba.cut(a))

In [ ]:
def poem(s):
    s = s.replace("\r", "").replace("\n", "")
    return " ".join(jieba.cut(s))
traindf["內容"] = traindf["內容"].apply(poem)
traindf

In [ ]:
testdf["內容"] = testdf["內容"].apply(poem)
testdf

In [ ]:
"""
由於 scikit-learn 不接受字串，所以我們一定要把類別轉換成整數
使用astype 轉成category obj
你可以使用 cat.categories 得到所有類別
再使用 cat.codes 轉換成整數

"""
traindf["作者"] = traindf["作者"].astype("category")
author_map = {cat:traindf["作者"].cat.categories.get_loc(cat)
        for cat in traindf["作者"].cat.categories}
author_map

{'李白': 0, '杜甫': 1, '白居易': 2}

In [ ]:
traindf["作者"] = traindf["作者"].cat.codes
traindf

In [ ]:
testdf["作者"] = testdf["作者"].replace(author_map)
testdf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
# 一定要使用 fit_transform才會幫你轉換成詞向量
# fit決定label多少種 transform counts label times
trainvec = vec.fit_transform(traindf["內容"])
print(vec.vocabulary_)  # 出現過的詞index 
print(trainvec)  # feature_vector

In [ ]:
# 由於我們用剛剛的vec訓練他,所以維度會保持跟剛剛一樣
testvec = vec.transform(testdf['內容'])
print(testvec)

In [ ]:
# 只要是文字, 我們通常就會選擇 MultinomialNB 做公式
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 1).fit(trainvec, traindf["作者"])
clf

MultinomialNB(alpha=1)

In [ ]:
from sklearn.metrics import accuracy_score
predict = clf.predict(testvec)
print("預測:", list(predict))
print("正確標籤:", list(testdf['作者']))
print("Naive-Bayes 正確率: ", accuracy_score(testdf['作者'], predict) * 100, "%")

預測: [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Naive-Bayes 正確率:  80.0 %


In [ ]:
s = input("來一首詩吧")
s = [poem(s)]
res_vec = vec.transform(s)
author = {index: author for author,index in author_map.items()}
print(author)
print("作者可能是:", author[clf.predict(res_vec)[0]])

來一首詩吧我喜歡吃牛排
{0: '李白', 1: '杜甫', 2: '白居易'}
作者可能是: 杜甫


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
vec = TfidfVectorizer()
trainbag = vec.fit_transform(traindf['內容'])
print("維度:", len(vec.get_feature_names_out()))
clf = MultinomialNB(alpha = 0.001)
clf.fit(trainbag, traindf['作者'])

維度: 52294


MultinomialNB(alpha=0.001)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
testbag = vec.transform(testdf['內容'])
predict = clf.predict(testbag)
print("預測:", list(predict))
print("正確標籤:", list(testdf['作者']))
print("Naive-Bayes 正確率: ", accuracy_score(testdf['作者'], predict) * 100, "%")

預測: [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Naive-Bayes 正確率:  80.0 %


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(testdf['作者'], predict)
pd.DataFrame(cm)


,0,1,2
0,8,1,1
1,0,8,2
2,1,1,8


In [ ]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=8)
clf = clf.fit(trainbag, traindf['作者'])
predict = clf.predict(testbag)
print("預測:", list(predict))
print("正確標籤:", list(testdf['作者']))
print("kNN 正確率: ", accuracy_score(testdf['作者'], predict) * 100, "%")


預測: [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 0, 1, 1, 2, 2, 0, 0, 2, 0, 2]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
kNN 正確率:  53.333333333333336 %


你最後發現，對於文1.6.1 優點
1. 啟動資料少的時候，單純貝氏還是可以達到一個不錯的結果
2. 對於文字這種稀疏的特徵，是一個非常好的分類器
1.6.2 缺點
1. 解釋性稍微低了一點
2. 用在普通非稀疏的模型，效果甚至可能不如 kNN 或者決策樹類型的演算法來得好字這種資料，還是單純貝氏來的比較合適
